In [213]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize
import time
from math import comb

## Group Assignment
### Team Number: 12
### Team Member Names: Sharuga, Derek, Alex
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.

CHECK DELIST:
- Purpose: checks that there is actually valid market data for this stock
- Code: if 'Close' not in data.columns or data['Close'].isnull().all():

PORTFOLIO GENERATOR
- Purpose: calculate how many shares of each stock to buy, and how much to pay in fees
- Code:

portfolio_value = 1000000 * weights  # Portfolio value based on optimized weights

shares = portfolio_val / np.array([ticker_prices[ticker] for ticker in selected_tickers])  # Number of shares

fee = np.minimum(3.95, 0.001 * shares)

portfolio_val -= fee


In [214]:
#October 01, 2023 to September 30, 2024, 2023

START_DATE = '2023-10-01'
END_DATE = '2024-11-20'

 # date on which we will take stock Close prices
TARGET_DATE = '2024-10-22'

INVESTMENT = 1000000

In [215]:
# read in CSV
csv = pd.read_csv('Tickers_Example.csv', header=None)

In [216]:
# filter out invalid stocks
def validity(tickers):

    # list to keep all of the valid stocks
    final_list = []

    for ticker in tickers:
        #only append the stock to the final stock list if it is isn't delisted, has a US or CAD currency, and has the required monthly share volume
        if (not check_delist(ticker) and
            check_currency(ticker) and
            check_volume(ticker)):
                final_list.append(ticker)
    return final_list

# checks if the consumed Ticker is delisted
def check_delist(ticker):
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period='1d')
        time.sleep(0.3)
        if data.empty:
            #if we can't find any data on the stock, it's delisted
            return True
        else:
            #check that there is actually valid market data for this stock
            if 'Close' not in data.columns or data['Close'].isnull().all():
                return True
            return False
    except Exception as e:
        #if there is an error in finding the stock's data, we can assume that it's delisted
        return True

# checks if the consumed Ticker meets the requirement of an average monthly share volume of 100,000 shares
def check_volume(ticker):
    volume = yf.Ticker(ticker).history(start=START_DATE, end='2024-09-30')['Volume']
    time.sleep(0.3)
    avg_monthly_volume = volume.resample('ME').mean()
    return avg_monthly_volume.mean() >= 100000

# checks whether or not the consumed Ticker is listed in CAD or in USD
def check_currency(ticker):
    stock = yf.Ticker(ticker)
    try:
        currency = stock.info.get('currency', None)
        return currency in ['USD', 'CAD']
    except Exception as e:
        return False  # if there's an error retrieving the data, we can assume it doesnt satisfy the required monthly volume

# create a new DataFrame only containing the valid Tickers in the consumed CSV
all_data = pd.DataFrame(validity(csv[0]))
all_data.rename(columns = {0:'Ticker'}, inplace=True)



$AGN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


In [217]:
# # fetch TSX60 index information
# tsx60_info = yf.Ticker('XIU.TO')
# tsx60 = pd.DataFrame(tsx60_info.history(start=START_DATE, end=END_DATE, interval='1d').Close)
# tsx60.index = pd.to_datetime(tsx60.index.strftime('%Y-%m-%d'))
# time.sleep(0.3)
# display(tsx60)

# # # fetch SP500 index information
# sp500_info = yf.Ticker('^GSPC')
# sp500 = sp500_info.history(start=START_DATE, end=END_DATE, interval='1d').Close
# sp500.index = pd.to_datetime(sp500.index.strftime('%Y-%m-%d'))

# # reindex sp500 so that it shares the same index as tsx60
# sp500 = sp500.reindex(tsx60.index, method='nearest')

# # calculate the average market return
# market_returns = (tsx60['Close'].pct_change() + sp500['Close'].pct_change())/2
# market_returns.index = pd.to_datetime(market_returns.index.strftime('%Y-%m-%d'))
# market_returns = market_returns.dropna()
# # market_returns_mean = market_returns.mean()

# # store the average market returns in a DataFrame
# market_returns_df = pd.DataFrame(market_returns)

# # display(market_returns_df.index)

In [218]:
# common_dates = market_returns_df.index.intersection(ticker_returns.index)

# market_returns_df = market_returns_df.loc[common_dates]
# ticker_returns = ticker_returns.loc[common_dates]

# months = market_returns_df.resample('MS').bfill().index

# for month in months:
#     temp = market_returns_df[(market_returns_df.index.month == month.month) & (market_returns_df.index.year == month.year)].index
#     if len(temp) < 18:
#         market_returns_df.drop(index=temp, inplace=True)
#         ticker_returns.drop(index=temp, inplace=True)

# # print([val for val in ticker_returns.index if val not in market_returns_df.index])

# # print(len(market_returns_df))
# # print(len(ticker_returns))

In [219]:
def days_to_drop(markets):

    markets.dropna(inplace=True)
    months = markets.resample('MS').bfill().index
    
    dates = []

    for month in months:
        temp = markets[(markets.index.month==month.month) & (markets.index.year==month.year)].index
        if len(temp) < 18:
            dates.extend(temp)
    
    return dates

    

In [220]:
ticker_returns = pd.DataFrame(columns=all_data['Ticker'])

# calculate and store all returns for all all Tickers
for ticker in all_data['Ticker']:
        data = yf.Ticker(ticker).history(start=START_DATE, end=END_DATE)['Close']
        time.sleep(0.3)
        ticker_returns[ticker] = data.pct_change().dropna()  # this code calculates daily returns
        
ticker_returns.index = pd.to_datetime(ticker_returns.index.strftime("%Y-%m-%d"))

ticker_returns


Ticker,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-03,-0.007770,-0.005532,0.000627,-0.006585,-0.023480,-0.036614,-0.029636,0.005803,-0.029588,-0.006645,...,-0.008191,-0.024235,-0.046045,0.006083,-0.009452,-0.016880,-0.009911,0.005626,-0.008576,-0.024891
2023-10-04,0.007309,0.001764,-0.001462,0.010319,0.012278,0.018281,0.010893,-0.011592,0.001158,-0.020067,...,-0.003086,0.009281,0.025412,0.032093,-0.002637,0.004579,0.002257,-0.007410,-0.000195,-0.003191
2023-10-05,0.007198,-0.001625,0.005750,0.000873,0.013309,-0.008189,0.007259,-0.002356,0.004626,-0.112628,...,-0.000455,0.007706,-0.009968,0.019378,0.011595,-0.018105,0.010987,-0.008059,-0.002212,0.020167
2023-10-06,0.014750,0.005358,0.007069,0.008170,0.011970,0.015878,0.015161,0.005851,0.000384,-0.009615,...,0.009836,-0.001477,0.024612,-0.003095,0.005164,0.012185,0.016621,0.009071,0.005803,-0.000314
2023-10-09,0.008451,0.005869,-0.001239,-0.000577,0.003614,0.002344,-0.003348,0.005924,0.009206,NaN,...,0.008297,NaN,NaN,NaN,NaN,-0.001783,0.003239,0.009088,0.000454,0.004708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-13,0.003969,-0.004325,-0.006012,0.026649,0.016431,0.024843,-0.003015,-0.035820,0.000218,0.014793,...,-0.018298,-0.001852,0.058715,-0.003199,0.002936,-0.033180,-0.014317,0.006279,0.027201,0.008077
2024-11-14,0.013770,-0.004227,-0.006134,-0.022014,-0.004468,-0.012237,0.002294,-0.013074,0.000654,-0.011662,...,0.022129,-0.002204,-0.048201,-0.021550,0.013747,0.001361,-0.020995,-0.020091,-0.007780,-0.016807
2024-11-15,-0.014109,-0.027354,0.007563,-0.023476,0.000264,-0.041942,-0.004960,0.014840,0.018518,-0.017699,...,-0.021162,-0.005347,-0.003650,0.002343,-0.010547,-0.023689,-0.001551,-0.000042,0.010781,-0.008149


In [221]:
def std_rank(returns):
    std_devs = returns.std()
    
    # Create a DataFrame with tickers and their respective standard deviations
    ranked_stocks = pd.DataFrame({
        "ticker": std_devs.index,
        "STD": std_devs.values
    })
    
    # Sort by standard deviation in ascending order (lowest STD at the top)
    ranked_stocks = ranked_stocks.sort_values(by="STD", ascending=True)
    
    return ranked_stocks['ticker'].values[:24].tolist()

In [222]:

ticker_lst = std_rank(ticker_returns)
print(ticker_lst)
index_lst = ["XIU.TO","^GSPC"]

exch_rate = yf.download("CADUSD=x",start="2024-11-20", end="2024-11-21")["Close"].iloc[0] #MUST BE CHANGED TO PULL NOVEMBER 22ND

#Takes a yf.Ticker "ticker", checks if the ticker is in canadian dollars. If so, it will just do a simple API pull for the price history.
#If the price is listed in USD, a conversion operation will simply be applied to each price.
def yfin_pull_convert_USD(ticker):
    listed_currency = ticker.fast_info["currency"]
    if listed_currency == "CAD":
        return ticker.history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
    if listed_currency == "USD":
        return ticker.history(start=START_DATE, end=END_DATE, interval="1d")["Close"] * (1/exch_rate)
    else: print("ticker currency is not in USD or CAD error")

#Test cases
#display(yf.Ticker("NVDA").history(start=START_DATE, end=END_DATE, interval="1d")["Close"]
#yfin_pull_convert_USD(yf.Ticker("NVDA"))

#Main function to store the stock prices of the ticker
def gen_tickers(tlist, ilist):
    ret_dataframe = pd.DataFrame()
    check_month_dataframe = pd.DataFrame()
    for i_str in ilist:
        index_ticker = yf.Ticker(i_str)
        check_month_dataframe[i_str] = yfin_pull_convert_USD(index_ticker)
        ret_dataframe[i_str] = yfin_pull_convert_USD(index_ticker)
    
    for t_str in tlist:
        ticker = yf.Ticker(t_str)
        ret_dataframe[t_str] = yfin_pull_convert_USD(ticker)
    ret_dataframe.drop(days_to_drop(check_month_dataframe), inplace=True)
    return ret_dataframe

#Function call: stores a dataframe of index values and stock values
stock_values = gen_tickers(ticker_lst,index_lst)
# display(stock_values)

[*********************100%***********************]  1 of 1 completed

['KO', 'RY.TO', 'PG', 'CL', 'T.TO', 'PEP', 'TD.TO', 'BK', 'LMT', 'ABT', 'UNP', 'MO', 'BLK', 'PM', 'MRK', 'AIG', 'AAPL', 'ABBV', 'ACN', 'UNH', 'AXP', 'BAC', 'PFE', 'BIIB']


In [223]:
market_label = "Market Returns"
#Function that transforms a list of values into a list of percent returns.
#Also adds a column of the simple average returns of the index tickers.
def convert_pct_returns(stock_value_df):
    ret_dataframe = stock_value_df.interpolate().pct_change()
    ret_dataframe.dropna(inplace=True)
    ret_dataframe[market_label] = ret_dataframe[index_lst].mean(axis=1)
    df_col_order = index_lst + ["Market Returns"] + ticker_lst
    ret_dataframe = ret_dataframe[df_col_order]
    return ret_dataframe

stock_returns = convert_pct_returns(stock_values)

display(stock_returns)

,XIU.TO,^GSPC,Market Returns,KO,RY.TO,PG,CL,T.TO,PEP,TD.TO,...,MRK,AIG,AAPL,ABBV,ACN,UNH,AXP,BAC,PFE,BIIB
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-03 00:00:00-04:00,-0.008913,-0.013744,-0.011329,-0.010815,-0.024235,0.002066,-0.004098,0.006083,-0.008689,-0.009452,...,-0.007606,-0.023480,-0.007770,-0.005532,-0.006585,-0.009911,-0.029636,-0.029588,-0.001179,-0.006130
2023-10-04 00:00:00-04:00,0.001384,0.008110,0.004747,0.002916,0.009281,0.003299,0.001277,0.032093,0.007215,-0.002637,...,0.003930,0.012278,0.007309,0.001764,0.010319,0.002257,0.010893,0.001158,-0.017404,0.023278
2023-10-05 00:00:00-04:00,0.005872,-0.001304,0.002284,-0.048329,0.007706,-0.014866,-0.018282,0.019378,-0.052158,0.011595,...,0.013996,0.013309,0.007198,-0.001625,0.000873,0.010987,0.007259,0.004626,0.004803,0.010149
2023-10-06 00:00:00-04:00,0.004464,0.011815,0.008140,0.014509,-0.001477,0.002017,0.002310,-0.003095,0.001187,0.005164,...,0.002703,0.011970,0.014750,0.005358,0.008170,0.016621,0.015161,0.000384,-0.010158,0.013397
2023-10-10 00:00:00-04:00,0.014359,0.011545,0.012952,0.016748,0.005309,0.004789,0.000864,0.005322,0.025641,0.009023,...,-0.002599,0.004929,0.005071,0.004385,0.000416,-0.001086,0.010782,0.036057,0.001207,0.003381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-25 00:00:00-04:00,-0.003760,-0.000299,-0.002030,-0.005646,-0.005724,-0.008254,-0.041408,-0.003623,-0.002149,-0.005714,...,-0.017945,-0.011071,0.003643,-0.009491,-0.009417,0.006687,-0.009741,-0.017820,-0.006634,-0.011802
2024-10-28 00:00:00-04:00,0.005392,0.002651,0.004022,-0.003736,0.012396,0.006420,0.001360,0.020455,-0.011351,-0.002043,...,0.002404,0.008592,0.008599,0.009742,0.001441,0.001204,0.015410,0.017427,0.014411,0.018273
2024-10-29 00:00:00-04:00,-0.001341,0.001614,0.000137,-0.016649,-0.001393,-0.012581,-0.010654,-0.005791,-0.013778,-0.010494,...,-0.004797,-0.010583,0.001157,-0.001213,0.004760,-0.005502,-0.003831,-0.002112,-0.013860,-0.008324


In [224]:
#Creating the correlation matrix: takes a dataframe of values and a string of columns to drop to make the correlation matrix
def correl(data, dropvalue):
    data_marketdrop = data.drop(labels=dropvalue, axis=1)
    ret_corr = data_marketdrop.corr()
    return ret_corr

stock_correlations = correl(stock_returns, [market_label] + index_lst)
#Apply a format to the DataFrame
display(stock_correlations.style.background_gradient(cmap='RdYlGn_r'))


,KO,RY.TO,PG,CL,T.TO,PEP,TD.TO,BK,LMT,ABT,UNP,MO,BLK,PM,MRK,AIG,AAPL,ABBV,ACN,UNH,AXP,BAC,PFE,BIIB
KO,1.000000,0.172835,0.500653,0.550128,0.236256,0.633000,0.064192,0.210403,0.364683,0.324544,0.319027,0.405345,0.202865,0.403203,0.190848,0.080744,0.067514,0.355467,0.185663,0.278368,0.113648,0.159303,0.253548,0.237976
RY.TO,0.172835,1.000000,0.049964,0.080668,0.360403,0.107992,0.345651,0.499717,0.114102,0.146647,0.413610,0.128738,0.520378,0.190217,0.095957,0.452997,0.169834,0.146356,0.171879,0.067017,0.467196,0.552770,0.295754,0.319764
PG,0.500653,0.049964,1.000000,0.644301,0.206853,0.502778,-0.080202,0.050251,0.153701,0.328424,0.100053,0.310603,0.032865,0.296432,0.376435,0.023242,-0.050055,0.281467,0.093679,0.184300,-0.020726,0.059640,0.088228,0.190282
CL,0.550128,0.080668,0.644301,1.000000,0.122302,0.471596,-0.020883,0.105151,0.135986,0.315081,0.179596,0.304637,0.129105,0.280047,0.277011,0.098584,-0.051470,0.245866,0.075938,0.192328,0.062193,0.061632,0.155339,0.125290
T.TO,0.236256,0.360403,0.206853,0.122302,1.000000,0.191570,0.217608,0.247272,0.121220,0.191675,0.258365,0.238174,0.272969,0.273807,0.143866,0.137964,0.087701,0.169166,0.254784,0.182153,0.108238,0.237257,0.212822,0.320230
PEP,0.633000,0.107992,0.502778,0.471596,0.191570,1.000000,0.020677,0.158151,0.301705,0.235880,0.144829,0.315698,0.079342,0.270047,0.165034,0.048584,0.031303,0.251740,0.116271,0.261102,0.049386,0.129132,0.139159,0.231436
TD.TO,0.064192,0.345651,-0.080202,-0.020883,0.217608,0.020677,1.000000,0.361355,0.113932,0.057234,0.242680,0.146488,0.317051,0.188653,0.080343,0.238764,0.067722,0.081020,0.171900,0.088696,0.373467,0.413507,0.279196,0.205184
BK,0.210403,0.499717,0.050251,0.105151,0.247272,0.158151,0.361355,1.000000,0.098817,0.185252,0.330289,0.150030,0.521852,0.241027,0.126798,0.463760,0.097776,0.182659,0.238223,-0.052184,0.487606,0.575557,0.235239,0.262511
LMT,0.364683,0.114102,0.153701,0.135986,0.121220,0.301705,0.113932,0.098817,1.000000,0.242430,0.191485,0.147521,0.028755,0.057458,0.002015,0.173738,-0.050852,0.122286,0.042889,0.156226,0.088445,0.124804,0.127665,0.054142
ABT,0.324544,0.146647,0.328424,0.315081,0.191675,0.235880,0.057234,0.185252,0.242430,1.000000,0.237333,0.140021,0.212908,0.273674,0.207327,0.015945,-0.054738,0.319316,0.177291,0.062985,0.036735,0.178749,0.182305,0.231577


In [225]:


#finding the ticker coordinates of the lowest correlation value (AI)
def low_correl_ticker_pairs(correlation_matrix):
    if correlation_matrix.shape[0] > 1:
        correl_pairs = correlation_matrix.unstack()
        correl_pairs = correl_pairs[correl_pairs.index.get_level_values(0) != correl_pairs.index.get_level_values(1)]
        lowest_corr_pair = correl_pairs.idxmin()
        lowest_corr_value = correl_pairs.min()
        return list(lowest_corr_pair)
    else:
        remaining_column_ticker = correlation_matrix.columns[0]
        return [remaining_column_ticker]

#recursing through the entire correlation matrix to extract every correlation pair
def corr_pair_extract(correl_matrix):
    #creating a list to store low-correlation pairs into
    low_corr_pairs = []
    while correl_matrix.shape[0] > 0:
        pair = low_correl_ticker_pairs(correl_matrix)
        #removing the tickers after they are packaged into a correlation pair
        if isinstance(pair, str):
            ticker1 = pair
            correl_matrix = correl_matrix.drop(ticker1, axis=0)
            correl_matrix = correl_matrix.drop(ticker1, axis=1)
        else: 
            low_corr_pairs.append(pair)
            correl_matrix = correl_matrix.drop(pair, axis=0)
            correl_matrix = correl_matrix.drop(pair, axis=1)
    return low_corr_pairs

#running the correlation pairing function and storing it in variable "stock pairs"
stock_pairs = corr_pair_extract(stock_correlations)
print(stock_pairs)

[['PG', 'TD.TO'], ['AAPL', 'ABBV'], ['BK', 'UNH'], ['LMT', 'MRK'], ['ABT', 'AIG'], ['PEP', 'AXP'], ['CL', 'BAC'], ['MO', 'ACN'], ['UNP', 'PFE'], ['KO', 'RY.TO'], ['BLK', 'PM'], ['T.TO', 'BIIB']]


In [226]:
#Arbitrarily assign a seed to keep results consistent
random.seed(21152764)

#This part creates lists of stocks, ranging from sizes 12 to 24, 5 of each size, which randomly selects stocks.
def make_port_list(stock_pairs):
    ret_list = []
    #iterates through the minimum selectable stock pairs (6) and the maximum (length of the stock pairs)
    print(len(stock_pairs))
    for i in range(6, len(stock_pairs)+1):
        print(i)
        #simple iteration, of 5 times
        for j in range(min(comb(len(stock_pairs), i), 5)):
            #generates a random list of indexes to pick from the stock pairs
            rand_list = random.sample(range(0,len(stock_pairs)), i)
            portfolio = []
            for k in rand_list:
                portfolio += stock_pairs[k]
            ret_list.append(portfolio)
    return ret_list

port_list = make_port_list(stock_pairs)

12
6
7
8
9
10
11
12


In [227]:
stock_values_just_date = stock_values
stock_values_just_date.index = pd.to_datetime(stock_values_just_date.index)
stock_values_just_date.index = stock_values_just_date.index.date

# Display the updated DataFrame
display(stock_values_just_date)


,XIU.TO,^GSPC,KO,RY.TO,PG,CL,T.TO,PEP,TD.TO,BK,...,MRK,AIG,AAPL,ABBV,ACN,UNH,AXP,BAC,PFE,BIIB
2023-10-02,28.043186,5984.447960,75.099634,110.245743,196.451105,95.981256,20.027452,228.953310,75.579323,56.477149,...,139.411846,82.054636,240.968565,197.609011,421.622218,707.311640,205.287624,36.248025,44.041079,352.866103
2023-10-03,27.793228,5902.197278,74.287452,107.573982,196.857026,95.587888,20.149286,226.963846,74.864975,55.470311,...,138.351466,80.127961,239.096301,196.515923,418.845740,700.301306,199.203776,35.175518,43.989173,350.703074
2023-10-04,27.831684,5950.062652,74.504040,108.572334,197.506482,95.709975,20.795938,228.601433,74.667587,55.362917,...,138.895252,81.111797,240.843763,196.862541,423.167881,701.882103,201.373641,35.216244,43.223579,358.866757
2023-10-05,27.995117,5942.303590,70.903366,109.409035,194.570344,93.960160,21.198923,216.678060,75.533386,55.967017,...,140.839276,82.191283,242.577363,196.542604,423.537112,709.593338,202.835342,35.379154,43.431200,362.508991
2023-10-06,28.120094,6012.511272,71.932136,109.247398,194.962701,94.177206,21.133320,216.935202,75.923470,56.584549,...,141.219911,83.175113,246.155461,197.595659,426.997499,721.387246,205.910527,35.392731,42.990009,367.365346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-25,36.783108,8105.230979,93.386850,170.220001,234.750993,133.423745,22.000000,239.732917,78.300003,104.941587,...,145.104083,107.202301,322.577733,262.144663,503.496343,787.843414,373.086904,58.457489,39.094732,253.548378
2024-10-28,36.981453,8126.721541,93.037975,172.330002,236.258135,133.605156,22.450001,237.011696,78.139999,106.769699,...,145.452958,108.123336,325.351720,264.698409,504.222030,788.792344,378.836357,59.476204,39.658137,258.181427
2024-10-29,36.931870,8139.839103,91.488969,172.089996,233.285714,132.181751,22.320000,233.746231,77.320000,106.839468,...,144.755208,106.979027,325.728085,264.377450,506.622292,784.452359,377.385025,59.350609,39.108471,256.032366
2024-10-30,36.872364,8112.975730,91.991350,171.839996,231.876267,132.098025,22.100000,231.946046,77.389999,105.960307,...,146.290256,107.230206,320.751633,281.193228,483.638407,785.610648,382.478591,59.043602,39.190923,252.836660


In [228]:
# portfolio_data = []

# def portfolio_maker(tickers, weights):
#     df = stock_values[tickers].interpolate().pct_change()
#     df.dropna(inplace=True)
    
#     # Calculate weighted returns for each ticker
#     weighted_returns = pd.DataFrame()
#     for ticker, weight in zip(tickers, weights):
#         weighted_returns[ticker] = df[ticker] * weight
    
#     # Sum across all weighted returns to get portfolio return
#     df['Portfolio Returns'] = weighted_returns.sum(axis=1)

    
#     # Organize columns in desired order
#     # df_col_order = index_lst + ["Portfolio Returns"] + ticker_lst
#     # ret = ret[df_col_order]
    
#     return df


# def portfolio_generator():
#     global portfolio_data

#     # creating portfolios -- starting wtih 12 stocks, going upto 24 stocks
#     for i in range(len(port_list)):

#         # for the ith portfolio, select the first i tickers
#         selected_tickers = port_list[i]
        
#         # objective function -- will be used in optimization steps
#         # returns the tracking error between the portfolio's returns and the market's returns
#         def objective(weights):
#             return np.std(portfolio_maker(selected_tickers, weights)['Portfolio Returns'].values - stock_returns['Market Returns'].values)
#             # return np.std(np.dot(stock_returns[selected_tickers].mean(), weights) - stock_returns['Market Returns'].dropna().mean())
        
#         # initial weights are calcualted so that, to start, every stock has equal weight
#         initial_weights = [1 / len(selected_tickers)] * len(selected_tickers)
        
#         # this constraint ensures that all of the stock weights sum to 1 (100%)
#         def sum_weights(weights):
#             return sum(weights) - 1
        
#         constraints = [{'type': 'eq', 'fun': sum_weights}]
        
#         # the bounds for each stock are that each stock must have a minimum weight of 1/2n% and maximum weight of 15%
#         bounds = [(1 / (2 * len(selected_tickers)), 0.15)] * len(selected_tickers)
        
#         # optimize the portfolio weights
#         result = minimize(objective, initial_weights, constraints=constraints, bounds=bounds)
        
#         if result.success:

#             weights = result.x
                        
#             # calculate portfolio value and the number of shares to purchase
#             portfolio_value = 1000000 * weights
#             shares = portfolio_value / np.array([stock_values_just_date.at[pd.to_datetime(TARGET_DATE).date(), ticker] for ticker in selected_tickers]) 

#             # calculate fees
#             fee = np.minimum(3.95, 0.001 * shares)
            
#             # subtract fees from portfolio value
#             portfolio_value -= fee
            
#             # Add to the results
#             portfolio_data.append({
#                 'tickers': (selected_tickers),  # Tickers in the portfolio
#                 'weights': weights,  # Optimized weights
#                 'Tracking Error': result.fun,  # Standard deviation (risk)
#                 'Difference of Mean Returns': abs(portfolio_maker(selected_tickers, weights)['Portfolio Returns'].values.mean() - stock_returns['Market Returns'].values.mean()),
#                 'num_stocks': i,  # Number of stocks in the portfolio
#                 'total_cost': portfolio_value.sum() + sum(fee),  # Total cost after fees
#                 'shares_purchased': shares,  # Shares purchased
#                 'portfolio_value': portfolio_value.sum(),  # Total portfolio value after fees
#                 'final_fees': sum(fee)
#             })

#     return pd.DataFrame(portfolio_data)

# portfolio_df = portfolio_generator()
# # portfolio_df['weights'].iloc[0]

# portfolio_df

In [229]:
np.std(np.dot(stock_returns[port_list[0]].values, [1/12]*12) - stock_returns['Market Returns'].values)

0.005695318162397172

In [230]:
portfolio_data = []

def portfolio_generator():
    global portfolio_data

    # creating portfolios -- starting wtih 12 stocks, going upto 24 stocks
    for i in range(len(port_list)):

        # for the ith portfolio, select the first i tickers
        selected_tickers = port_list[i]
        
        # objective function -- will be used in optimization steps
        # returns the tracking error between the portfolio's returns and the market's returns
        def objective(weights):
            return np.std(np.dot(stock_returns[selected_tickers].values, weights) - stock_returns['Market Returns'].values)
            # return np.std(np.dot(stock_returns[selected_tickers].mean(), weights) - stock_returns['Market Returns'].dropna().mean())
        
        # initial weights are calcualted so that, to start, every stock has equal weight
        initial_weights = [1 / len(selected_tickers)] * len(selected_tickers)
        
        # this constraint ensures that all of the stock weights sum to 1 (100%)
        def sum_weights(weights):
            return sum(weights) - 1
        
        constraints = [{'type': 'eq', 'fun': sum_weights}]
        
        # the bounds for each stock are that each stock must have a minimum weight of 1/2n% and maximum weight of 15%
        bounds = [(1 / (2 * len(selected_tickers)), 0.15)] * len(selected_tickers)
        
        # optimize the portfolio weights
        result = minimize(objective, initial_weights, constraints=constraints, bounds=bounds)
        
        if result.success:

            weights = result.x
                        
            # calculate portfolio value and the number of shares to purchase
            portfolio_value = 1000000 * weights
            shares = portfolio_value / np.array([stock_values_just_date.at[pd.to_datetime(TARGET_DATE).date(), ticker] for ticker in selected_tickers]) 

            # calculate fees
            fee = np.minimum(3.95, 0.001 * shares)
            
            # subtract fees from portfolio value
            portfolio_value -= fee
            
            # Add to the results
            portfolio_data.append({
                'tickers': (selected_tickers),  # Tickers in the portfolio
                'weights': weights,  # Optimized weights
                'Tracking Error': result.fun,  # Standard deviation (risk)
                'Difference of Mean Returns': abs(np.dot(stock_returns[selected_tickers].values, weights).mean() - stock_returns['Market Returns'].values.mean()),
                'num_stocks': i,  # Number of stocks in the portfolio
                'total_cost': portfolio_value.sum() + sum(fee),  # Total cost after fees
                'shares_purchased': shares,  # Shares purchased
                'portfolio_value': portfolio_value.sum(),  # Total portfolio value after fees
                'final_fees': sum(fee)
            })

    return pd.DataFrame(portfolio_data)

portfolio_df = portfolio_generator()
# portfolio_df['weights'].iloc[0]

portfolio_df

,tickers,weights,Tracking Error,Difference of Mean Returns,num_stocks,total_cost,shares_purchased,portfolio_value,final_fees
0,"[BK, UNH, PEP, AXP, ABT, AIG, MO, ACN, UNP, PF...","[0.133053154762463, 0.041666666666666664, 0.04...",0.005189,1.780265e-04,0,1000000.0,"[1256.0194298519054, 52.39511397744403, 171.23...",999992.591640,7.408360
1,"[LMT, MRK, AAPL, ABBV, PEP, AXP, BLK, PM, UNP,...","[0.07018495103997283, 0.062387334898858185, 0....",0.004535,1.456491e-05,1,1000000.0,"[87.16727117939918, 419.2243435739906, 456.230...",999992.496223,7.503777
2,"[UNP, PFE, KO, RY.TO, BLK, PM, BK, UNH, PEP, A...","[0.09154064345030577, 0.041666666666666664, 0....",0.004966,3.119479e-04,2,1000000.0,"[271.6795506765802, 1051.3747317210727, 505.97...",999993.942959,6.057041
3,"[BLK, PM, CL, BAC, T.TO, BIIB, BK, UNH, LMT, M...","[0.15, 0.041666666666666664, 0.046090763252024...",0.005124,2.266985e-04,3,1000000.0,"[107.4013691344584, 227.21161290317272, 331.37...",999990.555309,9.444691
4,"[LMT, MRK, CL, BAC, UNP, PFE, ABT, AIG, KO, RY...","[0.041666666666666664, 0.06573797224805472, 0....",0.005280,1.641532e-04,4,1000000.0,"[51.74855262642314, 441.73963045952036, 299.56...",999988.465700,11.534300
5,"[KO, RY.TO, UNP, PFE, T.TO, BIIB, PEP, AXP, BL...","[0.03571428571428571, 0.15, 0.0492708996469033...",0.004234,2.765613e-04,5,1000000.0,"[368.50206967835385, 874.9236697985524, 146.22...",999992.616015,7.383985
6,"[PEP, AXP, MO, ACN, UNP, PFE, BLK, PM, PG, TD....","[0.03571428571428571, 0.12075153390655967, 0.0...",0.004653,5.611290e-05,6,1000000.0,"[146.7710519655797, 317.6550445096438, 511.235...",999994.047073,5.952927
7,"[CL, BAC, ABT, AIG, UNP, PFE, MO, ACN, PG, TD....","[0.03571428571428571, 0.07442909787928027, 0.0...",0.004240,2.257804e-05,7,1000000.0,"[256.7720285416497, 1260.8766127564713, 220.39...",999991.687085,8.312915
8,"[ABT, AIG, AAPL, ABBV, LMT, MRK, CL, BAC, UNP,...","[0.03571428571428571, 0.14955477253947763, 0.1...",0.004462,5.436156e-05,8,1000000.0,"[220.39671899108623, 1391.2673054311842, 456.2...",999989.635265,10.364735
9,"[ABT, AIG, KO, RY.TO, BLK, PM, PEP, AXP, BK, U...","[0.03571428571428571, 0.08554505932063411, 0.0...",0.004085,2.545057e-04,9,1000000.0,"[220.39671899108623, 795.8023816495262, 368.50...",999992.493004,7.506996


In [ ]:
Portfolio_Final = portfolio_df[portfolio_df['Difference of Mean Returns'] == portfolio_df['Difference of Mean Returns'].min()]
Portfolio_Final

In [233]:
output_csv = pd.DataFrame()

output_csv['Ticker'] = Portfolio_Final['tickers'].values[0]
output_csv['Shares'] = Portfolio_Final['shares_purchased'].values[0]
output_csv.set_index(keys='Ticker', inplace=True)

output_csv.to_csv("Stocks_Group_12.csv", sep='\t', encoding='UTF-8')

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

ALEX
- coded get_sector function
- edited validity function
- coded strategy to choose final portfolio

DEREK
- coded make_port_list, corr_pair_extract, low_correl_ticker_pairs, correl, convert_pct_returns functions
- wrote up strategy

SHARUGA
- coded validity, stock_rank, portfolio_generator functions
- 